# Kubernetes Basics

<p align=center><a href=https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/><img src=images/k8s.png width=500></a></p>

## Introduction
> <font size=+1>Kubernetes is an open-source platform for managing __containerized__ workloads __focused on declarative configuration and automation__</font>

In order to run multiple containers, a long docker run command or docker compose (preferable in this case) is required. However, when an application that needs to run tens or hundreds of containers is deployed, the health and working condition of every container must be assured, which is a difficult task. This is where Kubernetes (k8s) thrives. 

Kubernetes orchestrates containers. It is capable of spinning up many containers in a particular configuration. Additionally, if a container goes down for any reason, Kubernetes can initiate a new one as a replacement. 

## Benefits
Kubernetes has many benefits and features

- __Multiple-container deployment:__ Kubernetes enables the deployment of many containers across a cluster.
    - The user chooses which and how many containers should be deployed.
    - The user also specifies the number of container replicas for fault tolerance.
- __Self-healing:__ If a container is down, i.e. not responding to health checks, Kubernetes will spin up a replacement.
- __Auto-scaling:__ If the given containers are not enough to process all the requirements, Kubernetes can deploy more containers within the cluster. Conversely, if the resources are in excess, Kubernetes can destroy some to maintain balance.
- __Discoverability:__ Each container can be accessed by its name (or IP) and port.
- __Load balancing:__ If a single node is overloaded by high traffic, Kubernetes distributes the load among several nodes.
- __Storage orchestration:__ Kubernetes allows us to mount storage volumes (per node, shared and others) to save/read data.

In this lesson, we cover
- [Kubernetes components](#kubernetes-components), which allow Kubernetes to orchestrate containers.
- [Kubernetes objects](#kubernetes-objects), which facilitates the deployment of resources.
- [How to install Kubernetes](#installing-kubernetes) and `kubectl`, which facilitates the creation of requests to the Kubernetes API.

## Kubernetes Components

> Kubernetes has __many__ concepts, and an adequate understanding of these concepts is essential for working with the platform efficiently.

When __Kubernetes__ is deployed, a cluster is created with the components shown in the diagram below:

![](./images/components-of-kubernetes.svg)

### High-level components overview

- __`Node`s__: worker machines (__either physical or virtual__) that host a containerised application.
- __`Pod`__: a component of the application; the __smallest deployable unit of work in `k8s`__. Pods consist of a group of one or more containers. These Pods are accessible to Kubernetes for container orchestration.
- __Control plane__: contains the Kubernetes' orchestrators that manage `node`s and `Pod`s.

> For learning, initially, we will work with a single `Node` on one physical machine.

> In a production environment, the __`Control Plane`__ can span multiple machines for increased fault tolerance, and there are usually multiple `Node`s (up to thousands).

### Control plane components

#### kube-apiserver

> __Exposes the `Kubernetes` API, which facilitates communication with the cluster.__

Communication with the cluster can be achieved via
- `http` requests, as the `kube-apiserver` provides a REST server (__read more about it [here](https://kubernetes.io/docs/concepts/overview/kubernetes-api/)__).
- __the command line__:
    - `kubectl` controls kubernetes cluster and workload.
    - `kubeadm` bootstraps kubernetes clusters (initialising it from config, tearing down, etc.).
      
> __Note that `command line` tools are almost exclusively preferred for interacting with the `kube-apiserver`.__

This is to ensure
- increased readability,
- easy automation,
- the easy handling of different internal API versions of `k8s`.

#### etcd

> __`etcd` stores Kubernetes objects that define the whole cluster.__

*Features*
- Data is stored locally on the control plane node(s).
- __If all the control planes go down, we lose the cluster state;__ consequently, the whole infrastructure must be recreated.

> `etcd` data should be backed up in an external storage device. For more information, check [here](https://kubernetes.io/docs/tasks/administer-cluster/configure-upgrade-etcd/#backing-up-an-etcd-cluster).

#### kube-scheduler

> __This handles newly created `pod`s and assigns them to a `node`.__

*Node selection*
The node is selected based on the
- resource requirements for the `pod`.
- applied policies/constraints.
- data locality (__move code towards data `spark`__).

For detailed information regarding pod scheduling, check out [this part of the documentation](https://kubernetes.io/docs/concepts/scheduling-eviction/assign-pod-node/).

#### kube-controller-manager

> This runs controller processes.

A controller process is a loop that watches the cluster (through `apiserver`) and __attempts to move the current state to the desired one.__

Each controller runs in a separate process. One example could be a `node controller` that responds when a node goes down.

#### cloud-controller-manager

This is similar to `kube-controller-manager`; however, it __watches the state of cloud resources.__

For example, the `node controller` queries the cloud provider to examine the health of nodes.

### Node components

Each node consists of the following:

#### kubelet

> __Ensures that appropriate containers are run within the `pod` and are healthy__, given a `PodSpec` (a specification that will be discussed later). 

#### kube-proxy

> Governs the network rules on nodes. __Enables two-way communication with `pod`s within and outside the cluster.__

#### Container runtime

> Software that is responsible for running containers.

Options include
- [`Docker`](https://docs.docker.com/engine/).
- [`containerd`](https://containerd.io/docs/), which is a simple container runtime (alternative to `Docker`).

For more information on projects that fulfill Kubernetes CRI specifications, check [here](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-node/container-runtime-interface.md) 

## Kubernetes Objects

> <font size=+1>Kubernetes objects are persistent entities in the `k8s` system that represent the DESIRED STATE of the cluster.</font>

Basically, Kubernetes resources are created from Kubernetes objects. 

### Creating Kubernetes objects

Here, we learn how to create these objects using declarative commands; however, imperative commands can also be used to manage objects (For more information on this, visit this [link](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/imperative-command/)).

Using objects, we can describe
- what applications to use and where (which node) to run these applications.
- the behaviour of the applications (e.g. when to restart, delete, upgrade, etc).
- what resources are available to the applications.

To create them, a request must be made to the __`kubelet-apiserver`'s__ __API.__

As mentioned previously, instead of directly sending REST requests, we can (and will) use `kubectl`. However, before _requesting_ the creation of resources, we must describe the object to be created.


### Describing Kubernetes objects

Kubernetes is a 'low-level' platform, on top of which other projects reside. Consider it as '_a programming language for deployment_'. Kubernetes employs __declarative programming__, meaning that the result is specified, rather than the steps required.

> <font size=+1> Kubernetes uses declarative programming to describe the state of the whole system. </font>

<br>

<details>
    <summary> <font size=+1>Click here to see the difference between imperative and declarative programming.</font> </summary>

<br>

> Using imperative programming (or configuration), __we can define each necessary step for achieving the desired result.__

Examples include the following:
- Create variable `x`, and assign it the value of `1`.
- Scrape data from a website, and retrieve an appropriate `<image>` tag.
- Pass the image through a function to calculate the probability that it contains a logo.

> Using declarative programming (or configuration), __we describe the desired state of a system without describing the required steps.__

Examples include the following:
- I want variable `x` with the value of `1` assigned to it.
- I want an `<image>` tag taken out of this website.
- I want the probability that the image contains a logo.

</details>

Since the API is RESTful, we can include a `JSON` description of the object.

__Although this approach is supported, it is difficult to maintain.__

> __An alternative approach is to use a more readable `YAML` to specify the objects, and allow `kubectl` to transform the representation into `JSON` format.__

An example object configuration, as well as a description of the fields, is provided below:

```
# configuration file stored in deployment_example.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  # Unique key of the Deployment instance
  name: deployment-example
spec:
  # 3 Pods should exist at all times.
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        # Apply this label to pods and default
        # the Deployment label selector to this value
        app: nginx
    spec:
      containers:
      - name: nginx
        # Run this image
        image: nginx:1.14
```

- `apiVersion`: specifies the `k8s` API version. Depending on the application to be deployed, you may need to use a different API version. 
- `kind`: the kind of object to be created (this will be explored in __Workloads__).
- `metadata`: uniquely defines the object, usually using the `name` field.

> <font size=+1>To determine the appropriate API version based on the resources to be used, check out Kuberenetes' documentation [here](https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.22/#-strong-api-overview-strong-). </font>

For example, in the declarative code above, a Deployment resource can be observed. In the documentation, you can check the Deployment API and determine that you would need the `apps/v1` API version.<br><br>

<p align=center><img src=images/Deployment.png></p>

### spec

> __`spec` describes the DESIRED characteristics of the resource.__

This is provided by the user, and, in this case, informs the `kubernetes` platform
- of every `Pod` with the label `app`, the name `nginx` and the kind `Deployment` (__described later__).
- that three replica `Pod`s should be run on the cluster.
- that each one should be created from a `template` that
    - assigns the `app` label with the value `nginx`.
    - `spec`ifies that it is created from a single container, which will be named `nginx` (for discoverability).
    - uses `image` `nginx:1.14`.

In the [API documentation](https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.22/#-strong-api-overview-strong-), you can view the available `spec`s for resources. 

<p align=center><img src=images/Deployment_Spec.png width=600></p>

Similarly, you can also view the parameters that can be passed to the `status` of the resource.

### status

> __`status` describes the CURRENT state of the resource.__

__While the user provides `spec`, `status` is updated internally by `kubernetes`.__

*Benefits of providing `status`*
- Allows users to query the current status of the deployment.
- __Allows controllers to act based on the current `status`__ and drive the objects towards the desired state.

### Desired vs actual state

> __It is worth noting that clusters may never reach the  `actual` state.__

Provided that `controllers` run continuously, everything should work well.

Discrepancies between the current and desired states may occur for a plethora of reasons, including

- frequent updates of the `config` files (`kubernetes` delays the implementation of changes).
- the delayed propagation of changes due to the significantly high number of nodes handled.
- random node failures (although `kubernetes` will reinstantiate the necessary `Pod`s).

### Creating and managing objects

Having described an object, the next step involves communicating to `kube-apiserver` to create it.

There are `three` approaches for doing this:

- __Imperative commands__: operates on existing objects __via commands__ (discouraged!).
- __Imperative object configuration__: operates on individual files.
- __Declarative object configuration__: operates on directories.

> __Important! These approaches should not be mixed, as it may result in `undefined behavior`.__

As mentioned above, we will focus on the `declarative` approach. For information on how to use imperative commands, visit this [link](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/imperative-command/).

### Declarative object configuration

> This involves specifying the configuration files, __but not the operation that should be performed.__

The `create`, `update` and `delete` operations are __automatically selected__ by the Kubernetes engine.

*Pros*
- __It is easy to work on directories of `config` files__; the changes will `apply` to all the files appropriately.
- __Different configs allow us to handle more objects.__
- __Only two commands are needed.__
- We do not have to replace the whole configuration; we only need to patch the required parts.

To confirm which operations will be applied, run

```
kubectl diff -f ./configs

kubectl diff -R -f ./configs  # Recursively find all `.yaml` files
```

Once we verify that the parsed configs drive the cluster to the desired state, one can run

```
kubectl apply -R -f ./configs # R for recursive
```

*Cons*
- They might be relatively difficult to debug (particularly when starting with `k8s`).
- High complexity of the operations behind the scenes.

This was a lot to process! Maybe a few examples can make everything a little bit more clear. Let's install Kubernetes to see it in action!

## Installing Kubernetes

Using `kubectl`, we can run commands to deploy Kubernetes resources on the cloud. Conversely, if you intend to deploy resources on your local machine and conduct experiments, use `minikube`. We will install both applications to help you learn how to deploy Kubernetes resources without worrying about launching cloud resources. Note, however, that we will consider some examples of how to deploy Kubernetes objects on the cloud.

### `kubectl`

`kubectl` allows you to deploy applications by setting up Kubernetes resources from the CLI. 

<details>
    <summary> For Linux Users </summary>

Simply run the following commands in your terminal:
```
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
curl -LO "https://dl.k8s.io/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl.sha256"
echo "$(<kubectl.sha256) kubectl" | sha256sum --check
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
```
If you encounter any error during the installation, refer to the [Kubernetes website](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/). 
</details>

<details>
    <summary> For Mac Users </summary>

Simply use `homebrew` for the installation:
```
brew install kubectl 
```
If you encounter any error during the installation, refer to the [Kubernetes website](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/) 
</details>

<details>
    <summary> For Windows Users </summary>

Install the latest release by running the following command:
```
curl -LO "https://dl.k8s.io/release/v1.22.0/bin/windows/amd64/kubectl.exe"
```
Move the directory containing `kubectl.exe` to your PATH.

>Important! By default, Docker Desktop comes with its own `kubectl` version. Ensure that the version you downloaded is above Docker in your PATH. For example, in our case, we have it installed in `C:\Users\yingy`; thus, it is above the Docker location.
<p align=center><img src=images/kubectl_path.png width=300></p>
</details>

Once installed, regardless of the OS, run the following command to ascertain that everything has been correctly installed:

`kubectl version --client`

More examples on `kubectl` will be provided in subsequent lessons. However, knowledge of the syntax is important:
```
kubectl [command] [TYPE] [NAME] [flags]
```
- `command`: specifies the operation to be performed (e.g. `get`, `describe`).
- `TYPE`: specifies the resource type; __it is case insensitive and works with abbreviations and plurals.__

For example, `kubectl get namespace` performs the same task as `kubectl get ns`. Additionally, `kubectl get pods` performs the same task as `kubectl get po`.

- `NAME`: specifies the name of the resource; __if omitted, it will return everything__ (e.g. `kubectl get pods`, as shown above).

The most useful ones are provided below. For the full list, check [here](https://kubernetes.io/docs/reference/kubectl/overview/#in-cluster-authentication-and-namespace-overrides).

There are two main commands when working with imperative commands:

- `kubectl annotate`, which can be described via `.yaml` config for a specific `type`.
- `kubectl create`, which should not be used (`kubectl apply` and declarative config files are preferred).

### Core commands


#### __apply__

> __Apply configuration change to a `resource`.__

Using this command, after starting `cluster` (e.g. with `minikube` for local development or `kubeadm` for real deployment), __we can configure the cluster to our liking.__

> This is the preferred management approach:

*Important flags*
- `-R`: __recursively read `config` files within a directory.__
- `-f`: filename (__or directory usually__) to read `config` from.

#### __diff__

`diff` checks what changes will be applied to the `cluster` desired state. This command should always be run before `apply`ing the configuration.

```
kubectl diff -f -R FILENAME [flags]
```


#### __get__

`get` lists one or more resources. It allows us to obtain basic information on `k8s` objects.

```bash
kubectl get (-f FILENAME | TYPE [NAME | /NAME | -l label]) [--watch] [--sort-by=FIELD] [[-o | --output]=OUTPUT_FORMAT] [flags]
```


#### __describe__

`describe` displays the state of one or more resources. It allows us to easily obtain an overview of the resource(s).

```bash
kubectl describe (-f FILENAME | TYPE [NAME_PREFIX | /NAME | -l label]) [flags]
```

As previously stated,
- the `file`(s) that describe the desired state can be specified __to obtain the current one.__
- the `TYPE`, `NAME` and other identifying resources can be specified for this command.


### Helpful debugging commands

#### __attach__

`attach` is used to read a container's `stdout` and/or interact with its `stdin`.

```bash
kubectl attach POD -c CONTAINER [-i] [-t] [flags]
```

It works similarly to `docker attach` and is __useful for debugging.__


#### __cp__

`cp` copies files from to/from containers.

```bash
kubectl cp <file-spec-src> <file-spec-dest> [options]
```

It is useful for debugging the disk output of container(s).


#### __exec__

This command `exec`utes a command against a `container` within a pod.

```bash
kubectl exec POD [-c CONTAINER] [-i] [-t] [flags] [-- COMMAND [args...]]
```

It is useful for debugging the container state.

### `minikube`

 [`minikube`](https://minikube.sigs.k8s.io/docs/start/) simplifies the process of learning Kubernetes from your local machine. If you already have Docker installed on your local computer, you should be able to install it.

<details>
    <summary> For Linux Users </summary>

Simply run the following commands in your terminal:
```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
```
</details>

 <details>
    <summary> For Mac Users </summary>

Simply run the following commands in your terminal:
```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-darwin-amd64
sudo install minikube-darwin-amd64 /usr/local/bin/minikube
```
If that does not work, use `homebrew`:
```
brew install minikube
```
_If minikube fails after installation via brew, you may need to remove the old minikube links and link the newly installed binary._
```
brew unlink minikube
brew link minikube
```
</details>
<details>
    <summary> For Windows Users </summary>

Download the .exe file here: https://storage.googleapis.com/minikube/releases/latest/minikube-installer.exe
After installing it, run PowerShell as an Administrator to add the binary to your `PATH`:
```
$oldPath = [Environment]::GetEnvironmentVariable('Path', [EnvironmentVariableTarget]::Machine)
if ($oldPath.Split(';') -inotcontains 'C:\minikube'){ `
  [Environment]::SetEnvironmentVariable('Path', $('{0};C:\minikube' -f $oldPath), [EnvironmentVariableTarget]::Machine) `
}
```
</details>

To confirm that it was correctly installed, run `minikube --help`. You may need to close and open the terminal before running any command.

Regardless of your OS, run `minikube start`. This will create a Kubernetes `config` file to inform Kubernetes (or `kubectl`) on where to start deploying the Kubernetes resources. __If you intend to deploy the resources to a cloud infrastructure, you can change this config file.__

To inspect the config context, run

In [4]:
!kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   default


Evidently, we are indeed using the cluster provided by `minikube`. In the configuration file (usually located in `~/.kube`), you can find this information and confirm that it is consistent with the information obtained:

<p align=center><img src=images/config_minikube.png width=500></p>

As can be observed above, the namespace is `default`. Namespaces are virtual clusters within physical clusters. They should be used for most tasks and to group semantically similar `k8s` objects. 

> <font size=+1>Namespaces provide a scope for names. The names of resources need to be unique within a namespace, but not across namespaces.</font>

By default, when you start `minikube` or when you change the kubeconfig file, kubernetes creates four namespaces. To view them, run the following command:

In [5]:
!kubectl get namespace

NAME              STATUS   AGE
default           Active   143m
kube-node-lease   Active   143m
kube-public       Active   143m
kube-system       Active   143m


To learn more about namespaces, visit this [link](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/). For now, know that each namespace contains different resources. You can view, for example, the pods inside the `kube-system` namespace:

In [8]:
!kubectl -n kube-system get pods -o wide

NAME                               READY   STATUS    RESTARTS       AGE    IP             NODE       NOMINATED NODE   READINESS GATES
coredns-78fcd69978-8j8hz           1/1     Running   0              170m   172.17.0.2     minikube   <none>           <none>
etcd-minikube                      1/1     Running   0              170m   192.168.49.2   minikube   <none>           <none>
kube-apiserver-minikube            1/1     Running   0              170m   192.168.49.2   minikube   <none>           <none>
kube-controller-manager-minikube   1/1     Running   0              170m   192.168.49.2   minikube   <none>           <none>
kube-proxy-w2pt7                   1/1     Running   0              170m   192.168.49.2   minikube   <none>           <none>
kube-scheduler-minikube            1/1     Running   0              170m   192.168.49.2   minikube   <none>           <none>
storage-provisioner                1/1     Running   1 (169m ago)   170m   192.168.49.2   minikube   <none>         

As mentioned previously, Kubernetes spins up resources that have gone down. See the below demonstration:

In [18]:
!kubectl -n kube-system delete pod kube-proxy-w2pt7 

pod "kube-proxy-w2pt7" deleted


If we observe the pods once more, we obtain the following:

In [20]:
!kubectl -n kube-system get pods

NAME                               READY   STATUS    RESTARTS   AGE
coredns-78fcd69978-8j8hz           1/1     Running   0          175m
etcd-minikube                      1/1     Running   0          175m
kube-apiserver-minikube            1/1     Running   0          175m
kube-controller-manager-minikube   1/1     Running   0          175m
kube-proxy-jb22b                   1/1     Running   0          77s
kube-scheduler-minikube            1/1     Running   0          175m
storage-provisioner                1/1     Running   0          13s


As shown, the resource is up and running again. Finally, Minikube provides a nice dashboard, which can be used to view the resources. It can initialize by running
```
minikube dashboard
```
The figure below displays the output after selecting `all namespaces` in the dropdown menu:

<p><img src=images/minikube_dashboard.png></p>

Observe all the resources on the left-hand side (Workloads, Service, Config and Storage, ...). In subsequent lessons, we will learn how to create them.

## Connecting to an AWS EKS

An Elastic Kubernetes Service can be created in Amazon to deploy your Kubernetes resources to a Cloud Infrastructure. This way, you can run your applications as well as AWS services in an orchestrated manner using clusters. If, however, you do not wish to deploy your Kubernetes resources to the cloud, you can skip the rest of this notebook and start working locally.

<font size=+1>__Before proceeding, please be advised that the following steps will require you to launch an instance that costs, at least, 0.0104 USD/h.__</font>

The first step involves creating an Amazon VPC that meets the EKS requirements. In your console, run the following command:

```
aws cloudformation create-stack \
  --region us-west-2 \
  --stack-name <your_stack_name> \
  --template-url https://amazon-eks.s3.us-west-2.amazonaws.com/cloudformation/2020-10-29/amazon-eks-vpc-private-subnets.yaml
```

In the example above, you will be creating the VPC in the us-west-2 region (Oregon); however, note that you can choose any EKS-supported region. For more information on this, visit this [link](https://docs.aws.amazon.com/general/latest/gr/eks.html). In this example, we will be using `my-eks-vpc-stack` as `<your_stack_name>`.

Now, an IAM role is required. You can create it from the console or the AWS console. Here, we will do this on our local machine. In this [link](https://aicore-files.s3.amazonaws.com/Cloud-DevOps/eks_policy.json), you will find a JSON file called `eks_policy.json`. This policy allows EKS to make calls to other AWS services. We can create the role using this policy.
```
aws iam create-role --role-name <your_role_name> --assume-role-policy-document file://"eks_policy.json"
```
In the example, we are using `AiCoreEKSClusterRole` as `<your_role_name>`. Now, attach this policy to the role:
```
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonEKSClusterPolicy --role-name <your_role_name>
```
Ensure that the role name is consistent.


Go to your AWS portal and search for EKS.

<p align=center><img src=images/EKS.png width=600></p>

Thereafter, click on add cluster. Ensure that you are in the right region. In the example, we established the policy in us-west-2, which corresponds to Oregon.

<p align=center><img src=images/EKS_2.png width=600></p>

In the next window, assign a name to your cluster, and select the newly created IAM role:

<p align=center><img src=images/EKS_3.png width=600></p>

Click on Next, and in the `specify networking section`, select the VPC you created:
<p align=center><img src=images/EKS_4.png width=600></p>

Leave the remaining values in their default settings, and click Create. 

Once created, you can use the `aws` command in your CLI; type the following:
```
aws eks --region <your-region> update-kubeconfig --name <name_of_your_cluster>
```
This will automatically change your config file (`~/.kube`). After running it, the output should be similar to that shown below:

In [1]:
!kubectl config get-contexts

CURRENT   NAME                                                        CLUSTER                                                     AUTHINFO                                                    NAMESPACE
*         arn:aws:eks:us-west-2:168573745887:cluster/aicore-cluster   arn:aws:eks:us-west-2:168573745887:cluster/aicore-cluster   arn:aws:eks:us-west-2:168573745887:cluster/aicore-cluster   
          minikube                                                    minikube                                                    minikube                                                    default


At this point, permission must be granted to the Kubernetes service accounts to access the resources. Thus, we need to create an IAM OpenID Connect (OIDC) provider. This will enable the Kubernetes service accounts to access the AWS resources.

To do so, go to your cluster, click `Configuration` and subsequently `Details`.

<p align=center><img src=images/EKS_5.png width=700></p>

Copy the OpenID Connect provider URL, and go to the IAM console: [https://console.aws.amazon.com/iam/](https://console.aws.amazon.com/iam/).

In the Identity Providers section, click on Add Provider.

<p align=center><img src=images/EKS_6.png width=700></p>

- For Provider Type, choose OpenID Connect.

- For Provider URL, paste the OIDC provider URL for your cluster, and thereafter, choose Get thumbprint.

- For Audience, enter `sts.amazonaws.com`, and choose Add provider.


If you intend to run AWS services, you need to create a Node. There are two main options: Fargate and Managed nodes.
1. Fargate: For running applications on AWS Fargate.
2. Managed nodes: For running applications on EC2 instances.

In this lesson, we will create a managed node. However, for information on how to create a Fargate node, visit this [webpage](https://aws.amazon.com/es/fargate/).

First, create a new role for the VPC Container Network Interface (CNI) plugin for Kubernetes. This plugin assigns an IP address from your VPC to each pod. 
Go to this [link](https://aicore-files.s3.amazonaws.com/Cloud-DevOps/cni_policy.json), and find a `cni_policy.json` file with the following content:
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Federated": "<Your ARN>"
      },
      "Action": "sts:AssumeRoleWithWebIdentity",
      "Condition": {
        "StringEquals": {
          "<Your Provider>:sub": "system:serviceaccount:kube-system:aws-node"
        }
      }
    }
  ]
}
```

Make sure you replace `<Your ARN>` and `<Your Provider>` with the corresponding credentials from the OIDC you created:

<p align=center><img src=images/EKS_7.png width=700></p>

In the console, run the following command to create a new role:
```
aws iam create-role --role-name <role_name> --assume-role-policy-document file://"cni_policy.json"
```
In this example, we substituted `<role_name>` for `AiCoreEKSCNIRole`. Now, attach the CNI policy to the created role:
```
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonEKS_CNI_Policy --role-name <role_name>
```
Next, associate the Kubernetes service account used by the VPC CNI with the IAM role you just created:
```
aws eks update-addon --region <your_region> --cluster-name <your_cluster> --addon-name vpc-cni --service-account-role-arn arn:aws:iam::<your_account_number>:role/<role_name>
```



You need to create a node IAM role to enable the AWS EKS `kubelet` make calls on our behalf. Use the `node_policy.json` file in this [link](https://aicore-files.s3.amazonaws.com/Cloud-DevOps/node_policy.json):
```
aws iam create-role --role-name <node_role_name> --assume-role-policy-document file://"node_policy.json"
```
As shown, we used `AiCoreEKSNodeRole` for the `<noe_role_name>`. As always, attach the necessary policies to this role:
```
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonEKSWorkerNodePolicy --role-name <node_role_name>

aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly --role-name <node_role_name>
```

In your cluster, go to `Configuration` and thereafter `Compute` to add a Node Group:
<p align=center><img src=images/EKS_8.png width=600></p>

Assign a name to your node, and select the IAM role you created:

<p align=center><img src=images/EKS_9.png width=600></p>

The paid instance will be deployed in the `Set compute and scaling configuration` section. You can use an instance of your choosing. In this example, however, we used the `t3.medium`. <font size=+1>Once the demonstration is over, make sure to `clean` everything up.</font>

Note that you will need a key pair in your specified region. You can quickly create one using the following command:
```
aws ec2 create-key-pair --region <your_region> --key-name <key_name> --output text > <key_name>.pem
```
In this example, we used `aicore-key` as the `<key_name>`.

Go back to the Node configuration. In `Specify networking`, enable the `Configure SSH access to nodes`, and select your key pair.

<p align=center><img src=images/EKS_10.png width=700></p>

Once created, you will be able to see the resources in your cluster, as well as the Workload deployed:

<p align=center><img src=images/EKS_10.png width=600></p>
<p align=center><img src=images/EKS_11.png width=600></p>

Inside each node, you can view the pods that have been created, as well as the events related to that node:

<p align=center><img src=images/EKS_13.png width=600></p>
<p align=center><img src=images/EKS_14.png width=600></p>

At this point, you should be able to deploy your AWS applications in EC2 instances using clusters. In the subsequent notebooks, we will focus on learning how to use Kubernetes, for which we __will use Minikube.__

### Cleaning up

Before wrapping up, endeavour to clean up all the resources you set up. Simply delete the nodes and, subsequently, the cluster containing these nodes.

<p align=center><img src=images/EKS_15.png width=600></p>

Once deleted, you can delete the entire cluster.

Additionally, delete the VPC you created. To do this, go to https://console.aws.amazon.com/cloudformation, select the VPC you created and delete it. 
Do the same with the IAM role you created: https://console.aws.amazon.com/iam/

## Conclusion
At this point, you should have a good understanding of

- Kubernetes and its benefits.
- Kubernetes components and objects.
- how to install Kubernetes and connect to an AWS EKS.
- the commands associated with using Kubernetes.

## Further Reading

- Check out [Kubernetes API Conventions](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-architecture/api-conventions.md) for an in-depth understanding of Kubernetes Objects and the internal mechanisms of the project.
- Explore the alternatives to `minikube`. Read up on `kind` and `k3s`, as well as their advantages and disadvantages.
- Check out [Client Libraries](https://kubernetes.io/docs/reference/using-api/client-libraries/), which allow us to communicate with `kubelet-apiserver` via certain programming languages. 
- Check out the [Python Client Library](https://github.com/kubernetes-client/python/), and explore the tasks that can be automated using it.
- Check out [imperative commands way to manage `k8s` objects](https://kubernetes.io/docs/concepts/overview/working-with-objects/object-management/#imperative-commands); do not, however, __use it.__ Find out the disadvantages of using it.
- Read more about [TTL (time to live) Controller](https://kubernetes.io/docs/concepts/workloads/controllers/ttlafterfinished/).
- Read about [canary `Deployment`s](https://kubernetes.io/docs/concepts/cluster-administration/manage-deployment/#canary-deployments).
- Read about [`Job` template expansion](https://kubernetes.io/docs/tasks/job/parallel-processing-expansion/).
- Read about `DaemonSet` tolerations.